In [120]:
import numpy as np
from scipy.integrate import quad
from scipy import constants

Mj = 1.0
Mp = 10*Mj
orbit = 1.0 # AU
pc_to_cm = 3.085677581e18 # cm



In [121]:
# cgs unit
from scipy import constants
CC = constants.c*1e2
HH = constants.h*1e7
KK = constants.k*1e7
GG = constants.g*1e2
RR = constants.R*1e7
NA = constants.N_A
SIGMA = constants.sigma*1e3
AMU = 1.661e-24

In [122]:
def planck(wl, *args):
    """
    Returns the Planck function evaluated at wavelength wl and temperature temp.
    """
    wl = np.array(wl)
    temp, = args
    fac = HH * CC / (wl * KK * temp)
    return (2.0 * HH * CC**2 / (wl**5)) * (1. / (np.exp(fac) - 1.))

In [123]:
def planck_dIdnu(nu, *args):
    """
    Returns the Planck function evaluated at wavelength wl and temperature temp.
    """
    nu = np.array(nu)
    temp, = args
    fac = HH * nu / (KK * temp)
    return (2.0 * HH * nu**3 / (CC**2)) * (1. / (np.exp(fac) - 1.))

In [124]:
def planck_photon(wl, *args):
    """
    Returns the thermal intensity of photons at wavelength wl and temperature temp.
    """
    wl = np.array(wl)
    energy_intensity = planck(wl, *args)
    energy_per_photon = HH * CC / wl
    return energy_intensity / energy_per_photon

In [125]:
wls = [1.e-6, 2.e-6]
temp = 300.
print planck(wls, temp)
print planck_photon(wls, temp)

[ 0.  0.]
[ 0.  0.]


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__main__.py:8: RuntimeWarning: overflow encountered in exp


In [126]:
# check with Solar Constant
Rsun_to_AU = 0.0046
print quad(planck, 0.01e-4, 5.0e-4, args=(5800.0,))[0]*Rsun_to_AU**2*np.pi


print quad(planck_dIdnu, 3e13, 3e+18, args=(5800.0,))[0]*Rsun_to_AU**2*np.pi

1350778.498
1356840.56849


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__main__.py:8: RuntimeWarning: overflow encountered in exp


In [127]:
# accretion temperature
Tacc=1e5*(Mp/Mj) # K

# accretion luminosity
Luminosity = 1e25*(Mp/Mj)**3 # erg/sec

# "class B" recombination rate
beta = 2.74e-14*(Mp/Mj)**(-3.0/4.0) # cm^3 / sec

In [128]:
planck_integrated = quad(planck, 0, 1.0e-4, args=(Tacc))[0]*np.pi
print planck_integrated

5.67037176037e+19


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__main__.py:8: RuntimeWarning: overflow encountered in exp


In [129]:
# denoting that surface area by "surface", 
###  surface * planck_integrated = Luminosity
###  surface =  Luminosity  / planck_integrated 

surface = Luminosity  / planck_integrated 
print surface

176355280.087


In [130]:
Luminosity / ( SIGMA*Tacc**4 )

176355241.53349352

In [131]:
# Suppose photons with energy > 13.6 eV can ionize hydrogen

eV_to_Joule = 1.6e-19
eV_to_erg = eV_to_Joule * 1e7
wl_ionize = HH * CC / (13.6 * eV_to_erg )
print wl_ionize*1e7, "[nm]"

91.2888641208 [nm]


In [132]:
# photons per second

source = surface * np.pi * quad(planck_photon, 0, wl_ionize, args=(Tacc))[0]
print source, "[s^-1]"

2.66827340875e+37 [s^-1]


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__main__.py:8: RuntimeWarning: overflow encountered in exp


In [133]:
# DSS COMMENT: I recommend not using "magic numbers"
# https://en.wikipedia.org/wiki/Magic_number_(programming)#Unnamed_numerical_constants
# I assume the 5.0 is AU?  I don't know what the 1e3 or the 1.8e6 are, though.
# It would be easier to follow the code if you replace all magic numbers with named constants.
# You can use some judgment here, like the "**(-2)" exponent doesn't need to be named,
# but I would name nearly everything else -- even obvious things like 13.6, which could be
# named "Rydberg" or something similar.


# YF temporary comment
# "1.8e6" comes from equation (18) of the manuscript 
density = 1.8e6 * (orbit / 5.0 )**(-2) # [cm^{-3}]

In [134]:
# DSS COMMENT: Isn't the calculation up to here in MKS, not cgs?  Do we want AU_to_m instead?
# Not sure about this one....
AU_to_cm = 1.49597870700e13
stromgren = 3.0 / ( 4.0 * np.pi ) * ( source / ( density**2 * beta ) )**(1.0/3.0) / AU_to_cm
print stromgren, "[AU]"

0.0222333151794 [AU]


In [135]:
Mjupiter = 6.9911e9 # cm
AU = 1.495978707e13 # cm
Mjupiter_to_AU = Mjupiter / AU
print Mjupiter_to_AU

0.000467326170305


In [136]:
stromgren / Mjupiter_to_AU

47.57558337664553

In [137]:
erg_s_cm_Hz_to_Jy = 1e23
surface * planck_dIdnu(1e9, 1e6) / ( 4. * np.pi * (100.0*pc_to_cm )**2 ) * erg_s_cm_Hz_to_Jy 


4.5284472304198582e-24

In [ ]:
 planck_dIdnu(1e9, 1e6)